In [188]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import ipywidgets as widgets
from IPython.display import display

In [189]:
from IPython.display import display, HTML

display(HTML("""
<style>
    .widget-label { 
        font-size: 24px !important;
    }
    .slider-value { 
        font-size: 24px !important;
    }
    .widget-hslider .widget-readout {
        font-size: 22px !important;
    }
</style>
"""))

In [308]:
grid_size = 50000
circle_radius = 10000

def update_circle(angle_deg, frequency):
    angle_rad = np.radians(angle_deg)
    
    circle_x = circle_radius * np.cos(angle_rad)
    circle_y = circle_radius * np.sin(angle_rad)
    
    fig = plt.figure(figsize=(12, 5))
    gs = gridspec.GridSpec(1, 2, width_ratios=[1, 1], wspace=0.3) 

    map_ax = plt.subplot(gs[0, 0])
    map_ax.set_xlim(-grid_size / 2, grid_size / 2)
    map_ax.set_ylim(-grid_size / 2, grid_size / 2)
    map_ax.scatter(circle_x, circle_y, edgecolors='red', facecolors='none', linewidths=2, s=50, label=f"Source")

    r1x = 0
    r1y = -1000
    dist_from_source_to_r1 = max(1, np.sqrt(((circle_x-r1x)**2) + ((circle_y-r1y)**2)))
    transmission_loss_to_r1 = 20*np.log10(dist_from_source_to_r1)
    map_ax.scatter(r1x, r1y, color='green', s=50, label=f"Receiver #1")

    r1x = 0
    r1y = 1000
    dist_from_source_to_r2 = max(1, np.sqrt(((circle_x-r1x)**2) + ((circle_y-r1y)**2)))
    transmission_loss_to_r2 = 20*np.log10(dist_from_source_to_r2)
    map_ax.scatter(r1x, r1y, color='blue', s=50, label=f"Receiver #2")
    map_ax.set_xlabel("X (meters)")
    map_ax.set_ylabel("Y (meters)")
    map_ax.set_title(f"Sound Reception (2 receivers; 1 source)")
    map_ax.legend()
    
    sampling_rate = 16000
    t = np.linspace(0, 30, sampling_rate*30)
    signal = np.sin(t*2*np.pi*frequency)

    start = 10
    duration = 10**(-np.floor(np.log10(frequency)))
    end = start+duration

    speed_of_sound_in_water = 1500
    time_to_r1 = dist_from_source_to_r1 / speed_of_sound_in_water
    time_to_r2 = dist_from_source_to_r2 / speed_of_sound_in_water

    gs_right = gridspec.GridSpecFromSubplotSpec(2, 1, height_ratios=[2,1], subplot_spec=gs[0, 1], hspace=0.8)

    received_ax = plt.subplot(gs_right[0])
    receive_signal1 = signal/(dist_from_source_to_r1**2)
    receive_signal2 = signal/(dist_from_source_to_r2**2)
    received_ax.plot(t+time_to_r1, receive_signal1+2e-8, 'green', label='rcvr #1')
    received_ax.plot(t+time_to_r2, receive_signal2-2e-8, 'blue', label='rcvr #2')

    chunk_of_received_signal1_in_window = receive_signal1[int((start-time_to_r1)*sampling_rate):int((end-time_to_r1)*sampling_rate)]
    chunk_of_received_signal2_in_window = receive_signal2[int((start-time_to_r2)*sampling_rate):int((end-time_to_r2)*sampling_rate)]

    received_ax.set_xlabel("Time (s)")
    received_ax.set_ylabel("Amplitude")
    received_ax.set_title("Received voltage signal")
    received_ax.legend(loc='upper right')
    received_ax.grid(which='both')
    received_ax.set_xlim(start, end)
    received_ax.set_ylim(-5e-8, 5e-8)

    summed_ax = plt.subplot(gs_right[1])
    receive_signal2 = signal/(dist_from_source_to_r2**2)
    summed_ax.plot(t[int((start)*sampling_rate):int((end)*sampling_rate)], chunk_of_received_signal1_in_window+chunk_of_received_signal2_in_window, 'k')
    summed_ax.set_xlabel("Time (s)")
    summed_ax.set_ylabel("Amplitude")
    summed_ax.set_title("Summed voltage signal")
    summed_ax.grid(which='both')
    summed_ax.set_xlim(start, end)
    summed_ax.set_ylim(-5e-8, 5e-8)

    plt.show()


In [309]:
angle_slider = widgets.IntSlider(
    value=0, min=0, max=360, step=1, description="Angle (°)", continuous_update=True, style={'description_width': 'initial'}, layout=widgets.Layout(width="1000px")
)

frequency_slider = widgets.IntSlider(
    value=0, min=1, max=1e3, step=1, description="Frequency (Hz)", continuous_update=True, style={'description_width': 'initial'}, layout=widgets.Layout(width="1000px")
)

interactive_plot = widgets.interactive(update_circle, angle_deg=angle_slider, frequency=frequency_slider)
display(interactive_plot)


interactive(children=(IntSlider(value=0, description='Angle (°)', layout=Layout(width='1000px'), max=360, styl…